In [1]:
import sympy as sp 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy import units as u
import datetime
import thesis_rcparams
import moon_tidal_deviation as MTD
from matplotlib.patches import Ellipse

# Spherical case
Try to show that the longest eigen vector of the tidal tensor will always align with the radial vector 

In [104]:
# Variables
x, y, z, G, M = sp.symbols('x y z G M')
r = sp.sqrt(x**2 + y**2 + z**2)

# Potential
Phi = -G*M/r

# Force = grad(Phi)
grad_Phi = [sp.diff(Phi, var) for var in (x, y, z)]

# Hessian (tidal tensor)
tidal_tensor = -sp.Matrix([
    [sp.diff(grad_Phi[i], var) for var in (x, y, z)]
    for i in range(3)
])

tidal_tensor

Matrix([
[3*G*M*x**2/(x**2 + y**2 + z**2)**(5/2) - G*M/(x**2 + y**2 + z**2)**(3/2),                                    3*G*M*x*y/(x**2 + y**2 + z**2)**(5/2),                                    3*G*M*x*z/(x**2 + y**2 + z**2)**(5/2)],
[                                   3*G*M*x*y/(x**2 + y**2 + z**2)**(5/2), 3*G*M*y**2/(x**2 + y**2 + z**2)**(5/2) - G*M/(x**2 + y**2 + z**2)**(3/2),                                    3*G*M*y*z/(x**2 + y**2 + z**2)**(5/2)],
[                                   3*G*M*x*z/(x**2 + y**2 + z**2)**(5/2),                                    3*G*M*y*z/(x**2 + y**2 + z**2)**(5/2), 3*G*M*z**2/(x**2 + y**2 + z**2)**(5/2) - G*M/(x**2 + y**2 + z**2)**(3/2)]])

Note that sympy is having a hard time doing all the algebraic manipulations, like finding the eigen vectors. I will simplify this expression to find the tidal tensor. This can be done by replacing $x^2 + y^2 + z^2$ with $r^2$ and factoring out  $\frac{GM}{r^3}$ This way, I can have a "scaled" tidal tensor.

Note that below I introduce $r$ without establishing that it is $r^2=x^2 + y^2 + z^2$. With SymPy, this manouver keeps the equation neat and orderly. 

In [14]:
r,x,y,z=sp.symbols('r x y z', real=True)
T = sp.Matrix(
    [1 - 3*x**2/r**2, 
     -3*x*y/r**2, 
     -3*x*z/r**2,
     -3*x*y/r**2,
     1 - 3*y**2/r**2,
     -3*y*z/r**2,
     -3*x*z/r**2,
     -3*y*z/r**2,
     1 - 3*z**2/r**2]
)
scaled_tidal_tensor=T.reshape(3,3)
scaled_tidal_tensor

Matrix([
[1 - 3*x**2/r**2,     -3*x*y/r**2,     -3*x*z/r**2],
[    -3*x*y/r**2, 1 - 3*y**2/r**2,     -3*y*z/r**2],
[    -3*x*z/r**2,     -3*y*z/r**2, 1 - 3*z**2/r**2]])

# Solve the equations of motion 
$$ \ddot{r}_\text{moon,earth} = -\nabla \Phi(r_\text{moon,earth})_{\text{Earth}} - T_\odot(r_\text{earth,sun}) \cdot \vec{r}_\text{moon,earth} - \vec{\omega} \times \left(\vec{\omega} \times {\vec{r}_{\text{moon,sun}}}\right)$$ 

We can use scipy's `solve_ivp`  since orbits are initial value problems of ordinary differential equations. Since I need to evaluate the position of the earth a function of time, the equations of motion will not be a set of autonomous differentia equations. They will depend on time 

In [ ]:
r,x,y,z=sp.symbols('r x y z', real=True)
T = -sp.Matrix(
    [1 - 3*x**2/r**2, 
     -3*x*y/r**2, 
     -3*x*z/r**2,
     -3*x*y/r**2,
     1 - 3*y**2/r**2,
     -3*y*z/r**2,
     -3*x*z/r**2,
     -3*y*z/r**2,
     1 - 3*z**2/r**2]
)
scaled_tidal_tensor=T.reshape(3,3)
scaled_tidal_tensor_func = sp.lambdify((r, x, y, z), scaled_tidal_tensor, "numpy")
